In [1]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain import HuggingFaceHub
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from PyPDF2 import PdfReader
from dotenv import load_dotenv
import os

from datasets import load_dataset

import cassio

d:\visionet\programs\langchain tutor\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
astra_db_token = os.getenv("Astra_DB_token")
astra_db_id = "ea60de45-be82-421c-aca8-e8aaafe31cc6"
huggingface_api_token = os.getenv("huggingface_token")

In [4]:
pdfReader = PdfReader("docs\LearningOpenCV_1_2.pdf")
pdfReader2 = PdfReader("docs\one-night-at-the-call-center-all-chapters.pdf")

In [5]:
raw_text = ''    
for i, pages in enumerate(pdfReader.pages):
    content = pages.extract_text()
    if content:
        raw_text += content

In [6]:
for i, pages in enumerate(pdfReader2.pages):
    content = pages.extract_text()
    if content:
        raw_text += content

In [7]:
raw_text

'Learning OpenCV\nGary Bradski and Adrian Kaehler\nBeijing · Cambridge · Farnham · Köln · Sebastopol · Taipei · Tokyo\nFM-R4886-AT1.indd   iFM-R4886-AT1.indd   i 9/15/08   4:26:38 PM9/15/08   4:26:38 PMLearning OpenCV\nby Gary Bradski and Adrian Kaehler\nCopyright © 2008 Gary Bradski and Adrian Kaehler. All rights reserved.\nPrinted in the United States of America.\nPublished by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions \nare also available for most titles ( safari.oreilly.com ). For more information, contact our \ncorporate/institutional sales department: (800) 998-9938 or corporate@oreilly.com .\nEditor:  Mike Loukides\nProduction Editor:  Rachel Monaghan\nProduction Services:  Newgen Publishing and \nData ServicesCover Designer:  Karen Montgomery\nInterior Designer:  David Futato\nIllustrator:  Robert Romano\nPrinting History: \nSeptember 2008: First 

In [8]:
cassio.init(token=astra_db_token, database_id=astra_db_id)

In [9]:
llm_t5 = HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-248M", huggingfacehub_api_token=huggingface_api_token)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

d:\visionet\programs\langchain tutor\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [10]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="MiniLM_embedd",
    session=None,
    keyspace=None
)

In [11]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len
)
chunks = text_splitter.split_text(raw_text)

Created a chunk of size 846, which is longer than the specified 800
Created a chunk of size 1126, which is longer than the specified 800
Created a chunk of size 1002, which is longer than the specified 800
Created a chunk of size 1070, which is longer than the specified 800
Created a chunk of size 817, which is longer than the specified 800


In [12]:
chunks[:10]

['Learning OpenCV\nGary Bradski and Adrian Kaehler\nBeijing · Cambridge · Farnham · Köln · Sebastopol · Taipei · Tokyo\nFM-R4886-AT1.indd   iFM-R4886-AT1.indd   i 9/15/08   4:26:38 PM9/15/08   4:26:38 PMLearning OpenCV\nby Gary Bradski and Adrian Kaehler\nCopyright © 2008 Gary Bradski and Adrian Kaehler. All rights reserved.\nPrinted in the United States of America.\nPublished by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions \nare also available for most titles ( safari.oreilly.com ). For more information, contact our \ncorporate/institutional sales department: (800) 998-9938 or corporate@oreilly.com .\nEditor:  Mike Loukides\nProduction Editor:  Rachel Monaghan',
 'corporate/institutional sales department: (800) 998-9938 or corporate@oreilly.com .\nEditor:  Mike Loukides\nProduction Editor:  Rachel Monaghan\nProduction Services:  Newgen Publishing and \nDat

In [13]:
len(chunks)

781

In [14]:
astra_vector_store.add_texts(chunks)
astra_vectr_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [15]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="MiniLM_embedd",
    session=None,
    keyspace=None
)

In [17]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vectr_index.query(query_text, llm=llm_t5).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


QUESTION: "what is the plot"
ANSWER: "The plot is about six people, three guys and three girls who worked as the Connex"

FIRST DOCUMENTS BY RELEVANCE:
    [0.7171] "on the diagram.
I tried to make sense of the chart and possibly connection to the st ..."
    [0.6860] "watch. It was 10:57 p.m. I hoped the call flow was moderate so the others  
wouldn’t ..."
    [0.6770] "page. It is a story about six people, three guys and three girls who worked as  
the ..."
    [0.6598] "it into a book.
‘Like a full book? Are you kidding? I can’t promise that. It’s a lot ..."

QUESTION: "who is the author"
ANSWER: "The author is Chetan Bhagat."

FIRST DOCUMENTS BY RELEVANCE:
    [0.7045] "it into a book.
‘Like a full book? Are you kidding? I can’t promise that. It’s a lot ..."
    [0.6876] "with his second book. In fact, he near as well wanted me to write the book for  
him ..."
    [0.6712] "It was completely dark.
‘So, pretty empty train,’ she said after ten minutes.
‘Yes,’ ..."
    [0.6679] "word